In [1]:
import matplotlib

matplotlib.use('TkAgg')
import LDP.basicDP.OLHbasic as lhb
import LDP.basicFunction.basicfunc as bf
import LDP.basicDP.LDPKVbasic as lkvb
import numpy as np
import random
import time

"""数据处理"""
# 关闭科学计数法显示
np.set_printoptions(suppress=True)

#### 读取文件
#### 构建kv对元组列表，每一行代表一个用户的kv对list

In [2]:
k_path='../Movie/data/Movie_k.txt'
v_path = '../Movie/data/Movie_v.txt'
lable_path = '../Movie/data/Movie_lable.txt'

data_k = bf.readtxt(k_path)
data_v = bf.readtxt(v_path)
data_l = bf.readtxt(lable_path)

n = len(data_k)  # 用户数量
l = len(data_l[0])  # key域的长度
epsilon = 5

# 构建kv对元组列表，每一行代表一个用户的kv对list
kv = []

for i in range(n):
    data_k[i]=list(map(int,data_k[i]))
    tmp = zip(data_k[i], data_v[i])
    kv.append(list(tmp))
  
# 构建label标签，我已经把标签规范化为[1,l]的数字了，也就是说不需要用之前的原始标签了
label=[i+1 for i in range(l)]

## 1.采样
#### 注意！！！！要转换为和samplek相同的格式，因为hash是专为str计算的，int和float转换得到的结果不同

In [3]:

samplekv = []
for i in range(n):
    random.seed(1)
    tmp = random.sample(kv[i], 1)
    samplekv.append(tmp[0])
samplekv_unzip = list(zip(*samplekv))
samplek = samplekv_unzip[0]

samplek

(1183,
 477,
 1019,
 164,
 361,
 52,
 1232,
 184,
 2598,
 953,
 844,
 36,
 47,
 590,
 50,
 582,
 477,
 1181,
 55,
 367,
 1004,
 1068,
 472,
 438,
 294,
 149,
 1109,
 264,
 156,
 263,
 4544,
 150,
 1010,
 140,
 1243,
 552,
 1389,
 194,
 36,
 1252,
 603,
 887,
 768,
 904,
 907,
 94,
 260,
 228,
 330,
 584,
 1070,
 159,
 443,
 1007,
 477,
 552,
 585,
 953,
 339,
 1126,
 1114,
 339,
 896,
 286,
 1728,
 320,
 908,
 207,
 286,
 476,
 3929,
 538,
 907,
 933,
 160,
 1018,
 908,
 505,
 896,
 62,
 904,
 4133,
 603,
 195,
 1059,
 1188,
 160,
 772,
 2349,
 330,
 900,
 1251,
 159,
 887,
 73,
 1632,
 580,
 2256,
 738,
 184,
 89,
 1010,
 3475,
 559,
 94,
 1196,
 1015,
 160,
 1014,
 603,
 997,
 588,
 64,
 93,
 374,
 1188,
 377,
 1172,
 152,
 149,
 1172,
 323,
 27,
 603,
 4626,
 52,
 159,
 164,
 1970,
 294,
 722,
 918,
 1265,
 714,
 725,
 1911,
 1018,
 1790,
 229,
 221,
 766,
 254,
 140,
 438,
 906,
 164,
 1063,
 886,
 184,
 997,
 346,
 588,
 536,
 892,
 896,
 1348,
 646,
 65,
 34,
 204,
 585,
 1173,
 

#### samplek的真实计数结果

In [4]:
count_true = []  # 真实计数结果
for i in range(l):
    count_true.append((label[i], samplek.count(label[i])))

count_true

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (6, 10),
 (7, 202),
 (8, 6),
 (9, 44),
 (10, 25),
 (11, 30),
 (12, 45),
 (13, 3),
 (14, 123),
 (15, 16),
 (16, 32),
 (17, 316),
 (18, 47),
 (19, 14),
 (20, 9),
 (21, 55),
 (22, 10),
 (23, 7),
 (24, 6),
 (25, 456),
 (26, 56),
 (27, 10),
 (28, 7),
 (29, 38),
 (30, 10),
 (31, 22),
 (32, 683),
 (33, 0),
 (34, 145),
 (35, 15),
 (36, 662),
 (37, 0),
 (38, 3),
 (39, 243),
 (40, 27),
 (41, 138),
 (42, 6),
 (43, 70),
 (44, 58),
 (45, 61),
 (46, 27),
 (47, 482),
 (48, 141),
 (49, 0),
 (50, 575),
 (51, 0),
 (52, 344),
 (53, 0),
 (54, 6),
 (55, 33),
 (56, 0),
 (57, 19),
 (58, 366),
 (59, 0),
 (60, 65),
 (61, 125),
 (62, 850),
 (63, 47),
 (64, 97),
 (65, 156),
 (66, 98),
 (67, 8),
 (68, 35),
 (69, 16),
 (70, 52),
 (71, 15),
 (72, 3),
 (73, 37),
 (74, 175),
 (75, 21),
 (76, 120),
 (77, 1),
 (78, 35),
 (79, 341),
 (80, 14),
 (81, 91),
 (82, 79),
 (83, 20),
 (84, 0),
 (85, 67),
 (86, 144),
 (87, 9),
 (88, 143),
 (89, 40),
 (90, 5),
 (91, 79),
 (92, 33),
 (

In [5]:
samplekpath='../Movie/result/samplekcount.txt'
#bf.savetxt(count_true,samplekpath)

保存文件成功


## 2.用OLH查找候选集

In [ ]:
starttime = time.clock()
print("开始执行OLH...")

# 调用OLH
est=lhb.OLH_1(epsilon,samplek,n,l)

# 程序运行结束时间
endtime = time.clock()
print('运行时间:', endtime - starttime)

est

开始执行OLH...


In [7]:
# 保存采样结果
bf.savetxt(est,'../Movie/result/sampleresult.txt')

保存文件成功


#### 生成top_k的候选集(候选集长度为2k)

In [8]:
k=20
est.sort(key=lambda x:x[1],reverse=True)
topk_candi=est[:2*k]


#在不会出现相同key的情况下，可以转为dict，提升速度
candi_dict=dict(topk_candi)
candi_k=list(candi_dict.keys())
candi_k

[149,
 164,
 588,
 152,
 160,
 844,
 524,
 477,
 603,
 1172,
 584,
 109,
 353,
 184,
 94,
 316,
 62,
 1114,
 1174,
 896,
 294,
 768,
 454,
 1018,
 1068,
 1173,
 908,
 1185,
 159,
 207,
 585,
 1170,
 738,
 50,
 1076,
 36,
 903,
 583,
 587,
 538]

In [9]:
# 保存候选集，长度为2k
#bf.savetxt(candi_k,'../Movie/result/candidate.txt')


保存文件成功


#### 与每个用户的项集kv取交集得到用户本地项集minikv

In [10]:
minik=[]
miniv=[]
minikv=[]
# 每个用户的交集项个数
numsize=[]
for i in range(n):
    # data_k[i]是每个用户的项集的k值
    # 将data_k[i]与候选集candi_k取交集得到tmpk
    tmpk=list(set(candi_k).intersection(set(data_k[i])))
    # 根据tmpk得到对应项的v值tmpv
    di=dict(kv[i])
    tmpv=list(map(di.get,tmpk))
    minik.append(tmpk)
    miniv.append(tmpv)
    minikv.append(list(zip(tmpk,tmpv)))
    numsize.append(len(tmpk))
    
numsize

[15,
 8,
 19,
 4,
 18,
 2,
 11,
 17,
 2,
 8,
 27,
 11,
 18,
 14,
 18,
 3,
 3,
 11,
 4,
 0,
 18,
 15,
 11,
 25,
 20,
 20,
 1,
 10,
 20,
 1,
 6,
 21,
 4,
 16,
 14,
 0,
 0,
 17,
 4,
 4,
 5,
 6,
 10,
 5,
 2,
 14,
 12,
 23,
 5,
 14,
 10,
 10,
 13,
 34,
 7,
 16,
 3,
 29,
 4,
 8,
 9,
 2,
 7,
 13,
 6,
 6,
 11,
 8,
 31,
 12,
 2,
 13,
 20,
 8,
 16,
 8,
 7,
 7,
 5,
 5,
 4,
 7,
 6,
 9,
 4,
 4,
 13,
 9,
 3,
 15,
 34,
 2,
 11,
 1,
 3,
 22,
 2,
 7,
 8,
 12,
 18,
 9,
 0,
 24,
 3,
 3,
 7,
 13,
 12,
 18,
 3,
 12,
 3,
 4,
 9,
 33,
 8,
 7,
 9,
 16,
 6,
 0,
 0,
 29,
 0,
 5,
 19,
 11,
 11,
 33,
 11,
 24,
 12,
 28,
 21,
 5,
 26,
 9,
 0,
 22,
 8,
 13,
 5,
 9,
 2,
 10,
 27,
 5,
 9,
 0,
 4,
 20,
 19,
 19,
 7,
 37,
 16,
 25,
 4,
 17,
 3,
 13,
 4,
 14,
 14,
 0,
 3,
 8,
 9,
 12,
 5,
 9,
 0,
 0,
 17,
 1,
 3,
 2,
 0,
 2,
 2,
 0,
 16,
 22,
 7,
 10,
 6,
 15,
 16,
 6,
 3,
 3,
 0,
 4,
 2,
 6,
 4,
 2,
 2,
 18,
 4,
 14,
 3,
 13,
 26,
 6,
 10,
 36,
 3,
 0,
 11,
 4,
 4,
 13,
 20,
 14,
 0,
 9,
 1,
 23,
 12,
 6,
 0,
 6,
 18,


In [11]:
# 保存用户取交集之后的本地项集minik和miniv
bf.savetxt(minik,'../Movie/result/minik.txt')
bf.savetxt(miniv,'../Movie/result/miniv.txt')

保存文件成功
保存文件成功


## 3.用OLH上传交集的项数

In [12]:
starttime = time.clock()
print("开始执行OLH...")
numsize_lable=[i for i in range(2*k+1)]
# 调用OLH
numsize_est = lhb.OLH(epsilon, numsize, n, numsize_lable)

# 程序运行结束时间
endtime = time.clock()
print('运行时间:', endtime - starttime)

numsize_est

开始执行OLH...
运行时间: 4.137952000000041


[(0, 9408.204),
 (1, 7189.516),
 (2, 7853.503),
 (3, 7942.574),
 (4, 7891.966),
 (5, 6978.983),
 (6, 5863.566),
 (7, 5727.935),
 (8, 5412.136),
 (9, 5442.501),
 (10, 5122.654),
 (11, 5316.991),
 (12, 5252.212),
 (13, 5049.777),
 (14, 4774.465),
 (15, 4533.568),
 (16, 3889.824),
 (17, 3725.852),
 (18, 3401.956),
 (19, 3181.301),
 (20, 2721.774),
 (21, 2495.046),
 (22, 2195.442),
 (23, 1984.91),
 (24, 1845.23),
 (25, 1626.6),
 (26, 1444.408),
 (27, 1361.41),
 (28, 1282.46),
 (29, 1302.704),
 (30, 1130.634),
 (31, 1033.465),
 (32, 920.101),
 (33, 820.908),
 (34, 594.181),
 (35, 598.23),
 (36, 450.452),
 (37, 436.282),
 (38, 298.626),
 (39, 280.407),
 (40, 278.382)]

In [ ]:
#### 获得L值

In [13]:
d=dict(numsize_est)
s=sum(list(d.values()))
target=0.9*s
target




125155.02240000003

In [24]:
# todo:其实这里的L值就是前面的90%的和，等下补个函数计算一下

#sum(list(d.values())[:25])
L=25

## 4.Padding and Sampling

In [25]:
padlabel=[i+1 for i in range(l,l+L)]
cp_label=candi_k+padlabel
upkv=[lkvb.P_S(minikv[i],l,L) for i in range(n)]

upkv

[(26750, -1),
 (26760, -1),
 (159, 1),
 (584, 1),
 (109, 1),
 (26767, 1),
 (26762, -1),
 (294, 1),
 (844, 1),
 (26756, 1),
 (353, 1),
 (26757, -1),
 (316, 1),
 (149, 1),
 (50, 1),
 (26749, 1),
 (26769, -1),
 (26752, 1),
 (94, 1),
 (26750, -1),
 (294, 1),
 (588, 1),
 (353, -1),
 (316, 1),
 (50, -1),
 (159, 1),
 (26764, 1),
 (160, 1),
 (149, 1),
 (26757, 1),
 (26746, -1),
 (50, 1),
 (26763, 1),
 (26753, 1),
 (152, 1),
 (26758, -1),
 (26759, -1),
 (584, 1),
 (26745, -1),
 (26765, -1),
 (26751, 1),
 (1174, -1),
 (26767, -1),
 (26751, 1),
 (26749, 1),
 (26756, -1),
 (1185, 1),
 (294, 1),
 (26755, -1),
 (26762, 1),
 (26756, 1),
 (26749, 1),
 (26767, -1),
 (908, 1),
 (26748, -1),
 (896, 1),
 (26757, 1),
 (768, -1),
 (26745, -1),
 (294, 1),
 (26757, -1),
 (26749, 1),
 (738, 1),
 (316, 1),
 (583, 1),
 (26766, -1),
 (26752, 1),
 (152, 1),
 (207, 1),
 (26752, 1),
 (26758, 1),
 (26769, -1),
 (26762, -1),
 (896, 1),
 (26750, 1),
 (26754, -1),
 (26752, 1),
 (26751, 1),
 (26753, -1),
 (26753, -1),
 (

In [26]:

#bf.savetxt(upkv,'../Movie/result/upkv.txt')
#bf.savetxt(cp_label,'../Movie/result/cp_label.txt')

保存文件成功
保存文件成功


#### UE扰动


In [27]:

d=2*k
eps=10
eps1=np.log((np.exp(eps)+1)/2)
eps2=eps
a=0.5
p=lkvb.p_values(eps2)
b=1-lkvb.p_values(eps1)

In [29]:
Y=lkvb.PCKV_UE(upkv,d,L,cp_label,a,p,b)

####聚合

In [30]:
f_k,m_k=lkvb.AEC_UE(Y,d,L,n,a,p,b)
print(f_k)
print(m_k)

[719, 444, 1034, 323, 335, 685, 793, 816, 673, 715, 742, 845, 1047, 283, 250, 1089, 285, 496, 661, 398, 1068, 591, 789, 447, 430, 497, 370, 697, 123, 323, 636, 473, 394, 756, 419, 299, 323, 598, 615, 467]
[233, 212, 224, 287, 133, 94, 136, 358, 156, 159, 246, 220, 286, 221, 198, 158, 97, 113, 129, 89, 262, 334, 188, 140, 106, 107, 104, 207, 158, 252, 267, 95, 72, 140, 163, 109, 102, 255, 343, 98]
[0.33922190107079658, 0.23233788673167638, 0.44971686184029247, 0.21572753315194823, 0.16445209384061354, 0.2767525278252973, 0.33091672428093255, 0.41938491182513676, 0.29480725997717572, 0.31105651891386626, 0.35222130822014902, 0.38002559573404177, 0.47679896006811012, 0.17745150098996601, 0.15723020097986218, 0.44574482076687921, 0.13339795453938269, 0.21536643850891066, 0.2807245688987105, 0.17131289205832734, 0.47571567613899735, 0.32947234570878225, 0.34824926714673582, 0.2074223563620842, 0.18900652956716821, 0.21356096529372282, 0.16661866169883896, 0.32188935820499331, 0.096927395592

In [31]:
lable_path = '../Movie/data/Movie_id_count_avg.txt'

id_count= bf.readtxt(lable_path)
id=[]
true_count=[]
true_avg=[]
for i in candi_k:
    tmp=id_count[int(i)-1]
    id.append(tmp[0])
    true_count.append(tmp[1])
    true_avg.append(tmpv[2])
true_count

[47777.0,
 33940.0,
 63299.0,
 32867.0,
 23628.0,
 41355.0,
 50054.0,
 59715.0,
 43272.0,
 45313.0,
 52244.0,
 53769.0,
 66172.0,
 24618.0,
 22104.0,
 63366.0,
 19381.0,
 33024.0,
 43295.0,
 24349.0,
 67310.0,
 47048.0,
 49581.0,
 30399.0,
 27635.0,
 32586.0,
 25253.0,
 46839.0,
 14965.0,
 27775.0,
 44208.0,
 29932.0,
 23220.0,
 47006.0,
 32685.0,
 20809.0,
 23445.0,
 41842.0,
 46054.0,
 30526.0]

In [32]:

# bf.savetxt(id,'../Movie/result/id.txt')
# bf.savetxt(true_count,'../Movie/result/true_count.txt')
# bf.savetxt(true_avg,'../Movie/result/true_avg.txt')

保存文件成功
保存文件成功
保存文件成功


In [33]:
est_count=[]
for i in f_k:
    est_count.append(i*n)
bf.savetxt(est_count,'../Movie/result/est_count.txt')


保存文件成功
